In [1]:
import sys

In [2]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/svu/e0384936/.ipython']

In [3]:
sys.path.append("/home/svu/e0384936/Anaconda3/Lib/site-packages")

In [4]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/svu/e0384936/.ipython',
 '/home/svu/e0384936/Anaconda3/Lib/site-packages']

In [5]:
%cd Strong_Baseline_of_Pedestrian_Attribute_Recognition

[Errno 2] No such file or directory: 'Strong_Baseline_of_Pedestrian_Attribute_Recognition'
/home/svu/e0384936/Pedestrian-Attribute-Recognition


In [6]:
import os
import pprint
from collections import OrderedDict, defaultdict

import numpy as np
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from batch_engine import valid_trainer, batch_trainer
from config import argument_parser
from dataset.AttrDataset import AttrDataset, get_transform
from loss.CE_loss import CEL_Sigmoid
from models.base_block import FeatClassifier, BaseClassifier
from models.resnet import resnet50
from tools.function import get_model_log_path, get_pedestrian_metrics
from tools.utils import time_str, save_ckpt, ReDirectSTD, set_seed

set_seed(605)

In [7]:
def main(args):
    visenv_name = args.dataset
    exp_dir = os.path.join('exp_result', args.dataset)
    model_dir, log_dir = get_model_log_path(exp_dir, visenv_name)
    stdout_file = os.path.join(log_dir, f'stdout_{time_str()}.txt')
    save_model_path = os.path.join(model_dir, 'ckpt_max.pth')

    if args.redirector:
        print('redirector stdout')
        ReDirectSTD(stdout_file, 'stdout', False)

    pprint.pprint(OrderedDict(args.__dict__))

    print('-' * 60)
    print(f'use GPU{args.device} for training')
    print(f'train set: {args.dataset} {args.train_split}, test set: {args.valid_split}')

    train_tsfm, valid_tsfm = get_transform(args)
    print(train_tsfm)

    train_set = AttrDataset(args=args, split=args.train_split, transform=train_tsfm)

    train_loader = DataLoader(
        dataset=train_set,
        batch_size=args.batchsize,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    valid_set = AttrDataset(args=args, split=args.valid_split, transform=valid_tsfm)

    valid_loader = DataLoader(
        dataset=valid_set,
        batch_size=args.batchsize,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    print(f'{args.train_split} set: {len(train_loader.dataset)}, '
          f'{args.valid_split} set: {len(valid_loader.dataset)}, '
          f'attr_num : {train_set.attr_num}')

    labels = train_set.label
    sample_weight = labels.mean(0)

    backbone = resnet50()
    classifier = BaseClassifier(nattr=train_set.attr_num)
    model = FeatClassifier(backbone, classifier)

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model).cuda()

    criterion = CEL_Sigmoid(sample_weight)

    param_groups = [{'params': model.module.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.module.fresh_params(), 'lr': args.lr_new}]
    optimizer = torch.optim.SGD(param_groups, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=False)
    lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=4)

    '''
    best_metric, epoch = trainer(epoch=args.train_epoch,
                                 model=model,
                                 train_loader=train_loader,
                                 valid_loader=valid_loader,
                                 criterion=criterion,
                                 optimizer=optimizer,
                                 lr_scheduler=lr_scheduler,
                                 path=save_model_path,
                                 measure='accuracy')

    print(f'{visenv_name},  best accuracy : {best_metric} in epoch{epoch}')
    '''
    
    best_metric2, epoch2 = trainer(epoch=args.train_epoch,
                                 model=model,
                                 train_loader=train_loader,
                                 valid_loader=valid_loader,
                                 criterion=criterion,
                                 optimizer=optimizer,
                                 lr_scheduler=lr_scheduler,
                                 path=save_model_path,
                                 measure='f1')
    print(f'{visenv_name},  best F1 score : {best_metric2} in epoch{epoch2}')


def trainer(epoch, model, train_loader, valid_loader, criterion, optimizer, lr_scheduler,
            path, measure):
    maximum = float(-np.inf)
    best_epoch = 0

    result_list = defaultdict()

    for i in range(epoch):

        train_loss, train_gt, train_probs = batch_trainer(
            epoch=i,
            model=model,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
        )

        valid_loss, valid_gt, valid_probs = valid_trainer(
            model=model,
            valid_loader=valid_loader,
            criterion=criterion,
        )

        lr_scheduler.step(metrics=valid_loss, epoch=i)

        train_result = get_pedestrian_metrics(train_gt, train_probs)
        valid_result = get_pedestrian_metrics(valid_gt, valid_probs)

        print(f'Evaluation on test set, \n',
              'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
                  valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
              'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
                  valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
                  valid_result.instance_f1))

        print(f'{time_str()}')
        print('-' * 60)

        if measure == 'accuracy':
            cur_metric = valid_result.ma
        elif measure == 'f1':
            cur_metric = valid_result.instance_f1

        if cur_metric > maximum:
            maximum = cur_metric
            best_epoch = i
            save_ckpt(model, path, i, maximum)

        result_list[i] = [train_result, valid_result]

    torch.save(result_list, os.path.join(os.path.dirname(path), 'metric_log.pkl'))

    return maximum, best_epoch

In [8]:
class args_config():
    def __init__(self):
        self.dataset="PETA"#"RAP"
        self.debug='store_false'
        self.batchsize=64
        self.train_epoch=30
        self.height=256
        self.width=192
        self.lr_ft=0.01
        self.lr_new=0.1
        self.classifier='base'
        self.momentum=0.9
        self.weight_decay=5e-4
        self.train_split="trainval" #choices=['train' 'trainval']
        self.valid_split="test" #choices=['test' 'valid']
        self.device=0
        self.redirector='store_false'
        self.use_bn='store_false'

In [9]:
args = args_config()

In [10]:
#args.dataset = "PETA"

In [11]:
vars(args)

{'dataset': 'PETA',
 'debug': 'store_false',
 'batchsize': 64,
 'train_epoch': 30,
 'height': 256,
 'width': 192,
 'lr_ft': 0.01,
 'lr_new': 0.1,
 'classifier': 'base',
 'momentum': 0.9,
 'weight_decay': 0.0005,
 'train_split': 'trainval',
 'valid_split': 'test',
 'device': 0,
 'redirector': 'store_false',
 'use_bn': 'store_false'}

In [12]:
main(args)

redirector stdout
OrderedDict([('dataset', 'PETA'),
             ('debug', 'store_false'),
             ('batchsize', 64),
             ('train_epoch', 30),
             ('height', 256),
             ('width', 192),
             ('lr_ft', 0.01),
             ('lr_new', 0.1),
             ('classifier', 'base'),
             ('momentum', 0.9),
             ('weight_decay', 0.0005),
             ('train_split', 'trainval'),
             ('valid_split', 'test'),
             ('device', 0),
             ('redirector', 'store_false'),
             ('use_bn', 'store_false')])
------------------------------------------------------------
use GPU0 for training
train set: PETA trainval, test set: test
Compose(
    Resize(size=(256, 192), interpolation=PIL.Image.BILINEAR)
    Pad(padding=10, fill=0, padding_mode=constant)
    RandomCrop(size=(256, 192), padding=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
trainval set:

  0%|          | 0/119 [00:00<?, ?it/s]

2020-06-21_12:56:51, Step 178/179 in Ep 0, 0.05s  train_loss:30.7670
Epoch 0, LR 0.1, Train_Time 37.80s, Loss: 42.8312


100%|██████████| 119/119 [00:32<00:00,  3.65it/s]


Evaluation on test set, 
 ma: 0.6743,  pos_recall: 0.4065 , neg_recall: 0.9421 
 Acc: 0.5925, Prec: 0.7451, Rec: 0.7206, F1: 0.7290
2020-06-21_12:57:24
------------------------------------------------------------
2020-06-21_12:57:30, Step 19/179 in Ep 1, 0.21s  train_loss:27.0500
2020-06-21_12:57:35, Step 39/179 in Ep 1, 0.21s  train_loss:25.3155
2020-06-21_12:57:39, Step 59/179 in Ep 1, 0.21s  train_loss:27.8440
2020-06-21_12:57:43, Step 79/179 in Ep 1, 0.21s  train_loss:27.2881
2020-06-21_12:57:47, Step 99/179 in Ep 1, 0.21s  train_loss:25.0905
2020-06-21_12:57:51, Step 119/179 in Ep 1, 0.21s  train_loss:26.0843
2020-06-21_12:57:55, Step 139/179 in Ep 1, 0.21s  train_loss:27.2845
2020-06-21_12:57:59, Step 159/179 in Ep 1, 0.21s  train_loss:30.0825


...

2020-06-21_13:30:45, Step 19/179 in Ep 29, 0.21s  train_loss:2.4441
2020-06-21_13:30:49, Step 39/179 in Ep 29, 0.21s  train_loss:2.7882
2020-06-21_13:30:54, Step 59/179 in Ep 29, 0.21s  train_loss:2.6304
2020-06-21_13:30:58, Step 79/179 in Ep 29, 0.21s  train_loss:2.3870
2020-06-21_13:31:02, Step 99/179 in Ep 29, 0.21s  train_loss:2.6194
2020-06-21_13:31:06, Step 119/179 in Ep 29, 0.21s  train_loss:2.9388
2020-06-21_13:31:10, Step 139/179 in Ep 29, 0.21s  train_loss:2.4944
2020-06-21_13:31:14, Step 159/179 in Ep 29, 0.21s  train_loss:2.6695


  0%|          | 0/119 [00:00<?, ?it/s]

2020-06-21_13:31:18, Step 178/179 in Ep 29, 0.05s  train_loss:11.1734
Epoch 29, LR 0.00010000000000000003, Train_Time 37.42s, Loss: 2.8839


100%|██████████| 119/119 [00:32<00:00,  3.62it/s]


Evaluation on test set, 
 ma: 0.8116,  pos_recall: 0.6516 , neg_recall: 0.9715 
 Acc: 0.8028, Prec: 0.8820, Rec: 0.8708, F1: 0.8750
2020-06-21_13:31:51
------------------------------------------------------------
PETA,  best F1 score : 0.8761075020077669 in epoch22
